In [5]:
#import librerie utili

import pandas as pd
import numpy as np

#import modelli di classificazione
from pgmpy.estimators import BicScore, HillClimbSearch, MaximumLikelihoodEstimator
from pgmpy.models import BayesianModel

#import Gaussian Naive Bayes model
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
#import di metrics per calcolare l'accuratezza del modello
from sklearn import metrics
from sklearn.model_selection import KFold

#Librerie per fare inferenza
from pgmpy.inference import VariableElimination

In [6]:
#preprocessing dei dati

#import del dataset
dataCoronary = pd.read_csv('coronary_prediction.csv')

#controllo dei valori nulli
dataCoronary.isnull().sum()

male                 0
age                  0
education          105
currentSmoker        0
cigsPerDay          29
BPMeds              53
prevalentStroke      0
prevalentHyp         0
diabetes             0
totChol             50
sysBP                0
diaBP                0
BMI                 19
heartRate            1
glucose            388
TenYearCHD           0
dtype: int64

In [7]:
#riempimento dei valori nulli di Education level con la moda
moda = dataCoronary['education'].mode()
dataCoronary['education'].fillna(1,inplace=True)

In [8]:
#riempimento dei valori nulli di cigsPerDay con moda se fumatore, 0 altrimenti

#visualizza la media delle sigarette al giorno per fumatori
dataC_smoker = dataCoronary.groupby(['currentSmoker'])
dataC_smoker['cigsPerDay'].describe()
#assegnazione della media ai fumatori, 0 per chi non fuma
dataCoronary['cigsPerDay'].fillna(9999,inplace=True)
index = []
for i,v in enumerate(dataCoronary['cigsPerDay']):
    if dataCoronary['cigsPerDay'].loc[i] == 9999:
        if dataCoronary['currentSmoker'].loc[i] == 1:
            dataCoronary['cigsPerDay'].loc[i] = 18
            index.append(i)
        else:
            dataCoronary['cigsPerDay'].loc[i] = 0
            index.append(i)

C:\Users\vetru\anaconda3\lib\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


In [9]:
#riempimento dei valori nulli di coloro che hanno una predisposizione all'hypertension

#controllo sui quale tipo di paziente utilizza medicine per la pressione
dataC_BPMed = dataCoronary.groupby(['prevalentStroke','prevalentHyp'])
dataC_BPMed['BPMeds'].sum()

#BPmeds è più probabile che un paziente con ipertensione utilizzi medicine per la pressione
dataCoronary['BPMeds'].fillna(9999,inplace=True)
index = []
for i,v in enumerate(dataCoronary['BPMeds']):
    if v == 9999:
        if dataCoronary['prevalentHyp'].loc[i] == 1:
            dataCoronary['BPMeds'].loc[i] = 1
            index.append(i)
        else:
            dataCoronary['BPMeds'].loc[i] = 0
            index.append(i)

In [10]:

#prendiamo da describe la media di totChol
dataCoronary.totChol.describe()
#assegnamo la media ai valori nulli
dataCoronary.totChol.fillna(236,inplace=True)

#BMI i valori nulli prendono il valore medio
dataCoronary.BMI.describe()
dataCoronary.BMI.fillna(25.8,inplace=True)

#heartrate valori nulli prendono il valore medio
dataCoronary.heartRate.describe()
dataCoronary.heartRate.fillna(75,inplace=True)

#glucose da vedere
dataCoronary['glucose'].fillna(9999,inplace=True)
index = []
for i,v in enumerate(dataCoronary['glucose']):
    if v == 9999:
        if dataCoronary['diabetes'].loc[i] == 1:
            dataCoronary['glucose'].loc[i] = 170
            index.append(i)
        else:
            dataCoronary['glucose'].loc[i] = 79
            index.append(i)

#fine riempimento valori nulli 
print(dataCoronary.isnull().sum()) 
 
#Convertiamo tutti i valori all'interno del dataFrame in interi
dataCoronary_int = np.array(dataCoronary, dtype = int)
dataCoronary = pd.DataFrame(dataCoronary_int, columns=dataCoronary.columns)

male               0
age                0
education          0
currentSmoker      0
cigsPerDay         0
BPMeds             0
prevalentStroke    0
prevalentHyp       0
diabetes           0
totChol            0
sysBP              0
diaBP              0
BMI                0
heartRate          0
glucose            0
TenYearCHD         0
dtype: int64


In [11]:
#K-FOLD dei classificatori Naive Bayes e Random Forest

kf = KFold(n_splits=5, shuffle=True, random_state= 100)

X_evaluation = dataCoronary.to_numpy()
y_evaluation = dataCoronary["TenYearCHD"].to_numpy()

gnb_metrics_score = {'accuracy_list': 0.0,
                    'precision_list' : 0.0,
                    'recall_list' : 0.0,
                    'f1_list' : 0.0 ,
}

rfc_metrics_score = {'accuracy_list': 0.0,
                    'precision_list' : 0.0,
                    'recall_list' : 0.0,
                    'f1_list' : 0.0 ,
}

gnb_cl = GaussianNB()
rfc_cl = RandomForestClassifier()

# ciclo della 5-fold cross validation
for train_index, test_index in kf.split(X_evaluation, y_evaluation):

    ##print(f"[+]Split {i}")
    ##i += 1
    training_set, test_set = X_evaluation[train_index], X_evaluation[test_index]

    #Dati di train
    data_train = pd.DataFrame(training_set, columns=dataCoronary.columns)
    X_train = data_train.drop("TenYearCHD", axis=1)
    y_train = data_train.TenYearCHD

    #Dati di test
    data_test = pd.DataFrame(test_set, columns=dataCoronary.columns)
    y_test = data_test.TenYearCHD
    X_test = data_test.drop("TenYearCHD", axis=1)

    #fit del classificatore Naive Bayes
    gnb_cl.fit(X_train, y_train)
    y_pred_gnb = gnb_cl.predict(X_test)

    #fit del classificatore RandomForest
    rfc_cl.fit(X_train, y_train)
    y_pred_rfc = rfc_cl.predict(X_test)

    #Salvo le metriche del fold attuale nel dizionario per il Naive
    gnb_metrics_score['accuracy_list'] = (metrics.accuracy_score(y_test,y_pred_gnb))
    gnb_metrics_score['precision_list'] = (metrics.precision_score(y_test,y_pred_gnb))
    gnb_metrics_score['recall_list'] = (metrics.recall_score(y_test,y_pred_gnb))
    gnb_metrics_score['f1_list'] = (metrics.f1_score(y_test,y_pred_gnb))

    #Salvo le metriche del fold attuale nel dizionario per il RandomForest
    rfc_metrics_score['accuracy_list'] = (metrics.accuracy_score(y_test,y_pred_rfc))
    rfc_metrics_score['precision_list'] = (metrics.precision_score(y_test,y_pred_rfc))
    rfc_metrics_score['recall_list'] = (metrics.recall_score(y_test,y_pred_rfc))
    rfc_metrics_score['f1_list'] = (metrics.f1_score(y_test,y_pred_rfc))

#Media delle metriche Naive
print("Media delle metriche del GaussianNB:")
print("Media Accuracy: %f" % (np.mean(gnb_metrics_score['accuracy_list'])))
print("Media Precision: %f" % (np.mean(gnb_metrics_score['precision_list'])))
print("Media Recall: %f" % (np.mean(gnb_metrics_score['recall_list'])))
print("Media f1: %f" % (np.mean(gnb_metrics_score['f1_list'])))
#Media delle metriche RandomForest
print("\nMedia delle metriche del RandomForest")
print("Media Accuracy: %f" % (np.mean(rfc_metrics_score['accuracy_list'])))
print("Media Precision: %f" % (np.mean(rfc_metrics_score['precision_list'])))
print("Media Recall: %f" % (np.mean(rfc_metrics_score['recall_list'])))
print("Media f1: %f" % (np.mean(rfc_metrics_score['f1_list'])))

Media delle metriche del GaussianNB:
Media Accuracy: 0.809917
Media Precision: 0.308511
Media Recall: 0.232000
Media f1: 0.264840

Media delle metriche del RandomForest
Media Accuracy: 0.850059
Media Precision: 0.428571
Media Recall: 0.048000
Media f1: 0.086331


In [12]:
#creazione classificatore e rete

X_train = dataCoronary.drop("TenYearCHD", axis=1)
y_train = dataCoronary.TenYearCHD

#Creazione e fitting del Naive Bayes
gnb = GaussianNB()
gnb.fit(X_train, y_train)

#creazione struttura rete
bic = BicScore(dataCoronary)
hc_bic = HillClimbSearch(dataCoronary)
bic_model = hc_bic.estimate(scoring_method=bic)
#Creazione della rete bayesiana e fit
bNet = BayesianModel(bic_model.edges())
bNet.fit(dataCoronary, estimator=MaximumLikelihoodEstimator) 

  0%|          | 13/1000000 [00:03<71:23:16,  3.89it/s]
C:\Users\vetru\anaconda3\lib\site-packages\pgmpy\models\BayesianModel.py:8: FutureWarning: BayesianModel has been renamed to BayesianNetwork. Please use BayesianNewtork class, BayesianModel will be removed in future.
  warnings.warn(


In [13]:
#test funzionamento classificatore con calcolo della probabilità

datiUtente = {
    "male": [1],
    "age": [45],
    "education": [4],
    "currentSmoker": [0],
    "cigsPerDay": [0],
    "BPMeds": [0],
    "prevalentStroke": [0],
    "prevalentHyp": [0],
    "diabetes": [1],
    "totCHol": [23],
    "sysBP": [106],
    "diaBP": [70],
    "BMI": [26.97],
    "heartRate": [80],
    "glucose": [77],
}
dati_utente = pd.DataFrame(datiUtente)

#calcolo probabilità tra malato o non in 10 anni
print("  NonMalato \t   Malato")
print(gnb.predict_proba(dati_utente))

  NonMalato 	   Malato
[[3.64416896e-06 9.99996356e-01]]


In [14]:
#calcoliamo la probabilità per un soggetto presumibilmente sano e uno malato.

dati = VariableElimination(bNet)
soggetto_sano = dati.query(variables = ['TenYearCHD'], evidence= {  'male' : 0, 'currentSmoker' : 1, 'cigsPerDay' : 30, 'diabetes' : 0, 'age' : 25  })
print('Probabilità per un soggetto potenzialmente sano: ')
print(soggetto_sano)

sommetto_malato = dati.query(variables = ['TenYearCHD'], evidence= {  'male' : 1, 'currentSmoker' : 1, 'cigsPerDay' : 30 ,'diaBP' : 120, 'BPMeds' : 1, 'diabetes' : 1, 'age' : 70, 'prevalentStroke' : 1 })
print('\nProbabilità per un soggetto potenzialmente malato:')
print(sommetto_malato)

C:\Users\vetru\anaconda3\lib\site-packages\pgmpy\factors\discrete\DiscreteFactor.py:529: UserWarning: Found unknown state name. Trying to switch to using all state names as state numbers
  warnings.warn(
Eliminating: prevalentHyp: 100%|██████████| 2/2 [00:00<00:00, 250.05it/s]

Probabilità per un soggetto potenzialmente sano: 
+---------------+-------------------+
| TenYearCHD    |   phi(TenYearCHD) |
+===============+===================+
| TenYearCHD(0) |            0.8755 |
+---------------+-------------------+
| TenYearCHD(1) |            0.1245 |
+---------------+-------------------+

Probabilità per un soggetto potenzialmente malato:
+---------------+-------------------+
| TenYearCHD    |   phi(TenYearCHD) |
+===============+===================+
| TenYearCHD(0) |            0.4485 |
+---------------+-------------------+
| TenYearCHD(1) |            0.5515 |
+---------------+-------------------+
